# dev measure birth/death events recursively

In [6]:
from lib.my_initialization import *
from lib.controller.controller_LR import get_one_step_explicit_synchronous_splitting
from lib.model.LR_model import *
from lib.utils.utils_traj import *
from lib.routines.bdrates import *
from lib.measure.utils_measure_tips_cpu import *
from lib.utils.stack_txt_LR import *
from lib.viewer import *
import trackpy
from lib.utils import load_buffer

#automate the boring stuff
# from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
    
%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart


## TODO: dev python routine that takes in a spiral tip in pdict and follows it iteratively to its death, zooming in as needed, until a certain absolute tolerance is found in a zero minimum range is reached.  
Consider dict_atol={'batch':1e-4,'test':1e-2,'precise':1e-6}



input:dict_topo,point,width,height
output:pid_lst within r distance of point

In [7]:
def get_particles_in_range(dict_topo,point,width,height,r):
    dict_topo['xy']
    
    return pid_lst_within_r

In [8]:
txt_fn=f'{nb_dir}/Data/test_data/ic008.33_t_218.8.npz'
t=218.8
dt=0.01
txt=load_buffer(txt_fn)
inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
width,height=txt.shape[:2]
print(txt.shape)

(200, 200, 18)


In [9]:
V_threshold=-50
jump_threshold = 40
ds=5.0
# get_one_step
__, arr39, one_step = get_one_step_explicit_synchronous_splitting(
    nb_dir,dt,width,height,ds,diffCoef=0.0005,Cm=1.0)

In [10]:
#the heavyweight spiral tip measures
comp_dict_tips=get_comp_dict_topo_full_color(width=width,height=height,level1=V_threshold,level2=0,
                                             jump_threshold=jump_threshold,ds=ds)
# comp_dict_tips=get_compute_all_spiral_tips(width, height, mode='simp')

In [11]:
#measure tips that are already present topologically (slower)
img=inVc[...,0];dimgdt=dVcdt[...,0]
# dict_topo=comp_dict_topo_simple(img,dimgdt,t)
dict_topo=comp_dict_tips(img,dimgdt,t,txt)
pid_lst_all=dict_topo['pid']
n_tips=len(pid_lst_all)
n_tips

4

In [12]:
dict_topo.keys()

dict_keys(['x', 'y', 's1', 's2', 't', 'pid', 'greater_pid', 'lesser_pid', 'greater_arclen', 'lesser_arclen', 'greater_arclen_values', 'lesser_arclen_values', 'greater_mean_V', 'lesser_mean_V', 'greater_mean_curvature', 'lesser_mean_curvature', 'greater_xy_values', 'lesser_xy_values', 'greater_V_values', 'lesser_V_values', 'greater_curvature_values', 'lesser_curvature_values'])

In [13]:
#initialize system
pdict=ParticlePBCDict(dict_tips=dict_topo, width=width, height=width)#, **kwargs)
t_prev=t;txt_prev=txt.copy()
# #zoom system
# txt,pdict=zoom_system(txt,pdict)
# txt,pdict=zoom_system(txt,pdict)
# dt=dt/10
# width,height=txt.shape[:2]
# comp_dict_topo_simple=get_comp_dict_topo_simple(width=width,height=height,level1=V_threshold,level2=0)
x_values,y_values,pid_values=pdict.get_current_locations()
inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
img=inVc[...,0];dimgdt=dVcdt[...,0]

In [14]:
comp_distance=get_distance_L2_pbc(width,height)

In [15]:
scale=ds/width
xy_values=np.array(tuple(zip(x_values,y_values)))
assert(xy_values[0].shape[0]==2)

In [16]:
#minimum distance of all pid found
item_lst=[]
for i in range(n_tips):
    for j in range(n_tips):
        if i > j:
            #the ith tip location
            point1=xy_values[i]
            #the jth tip location
            point2=xy_values[j]
            #their distance, in cm
            dist=comp_distance(point1,point2)
            #add to list of distances
            item=(dist,i,j)
            item_lst.append(item)
item_lst=sorted(item_lst)
item_lst

[(0.21228192704422172, 3, 2),
 (19.635486143424185, 1, 0),
 (85.31496830431027, 3, 0),
 (85.49533638494285, 2, 0),
 (96.08268845547843, 2, 1),
 (96.27196815000205, 3, 1)]

In [26]:
#identify the pairs to follow to existential event
num_pairs_to_follow=2
pid_pair_lst=[]
for item in item_lst[:num_pairs_to_follow]:
    d,i,j=item
    pid_pair=i,j
    pid_pair_lst.append(pid_pair)


In [27]:
#define initialization target points as
point_target_init_lst=[]
for pid_pair in pid_pair_lst:
    #the  mean location of these tips 
    point_target_init=np.mean((x_values[pid_pair[0]],x_values[pid_pair[1]])),np.mean((y_values[pid_pair[0]],y_values[pid_pair[1]]))
    point_target_init_lst.append(point_target_init)

In [28]:
#set range to some value
#the ith tip location
point1=np.array(point_target_init_lst[0])
#the jth tip location
point2=np.array(point_target_init_lst[1])
#their distance, in cm
r=comp_distance(point1,point2)*scale #cm distance between the targets
r

2.363878298954001

In [29]:

# def follows_pair_to_death(txt,pdict,pid_pair,atol,dt,ds):
# zooming in as needed, until a certain absolute tolerance is found in a zero minimum distance between tips is reached.  


# def zoom_in(txt,pdict,t_factor,s_factor):
# consider 
dict_atol={'batch':1e-4,'test':1e-2,'precise':1e-6}
atol=dict_atol['test']
## TODO: turn ^this into a python routine 

In [32]:
from lib.routines.return_min_distance import find_stopping_point
from inspect import getsource
# print(getsource(find_stopping_point))

In [ ]:
#TODO: determine which pid_pair dies first
# item=t_death,pid_pair

In [33]:
t_death_lst=[]
for pid_pair in pid_pair_lst:
#init txt
txt_fn=f'{nb_dir}/Data/test_data/ic008.33_t_218.8.npz'
t=218.8;dt=0.01
txt=load_buffer(txt_fn);width,height=txt.shape[:2]
inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
#initialize system
dict_topo=comp_tips()
pdict=ParticlePBCDict(dict_tips=dict_topo, width=width, height=width)#, **kwargs)
t_prev=t;txt_prev=txt.copy()
# #zoom system
# txt,pdict=zoom_system(txt,pdict)
# txt,pdict=zoom_system(txt,pdict)
# width,height=txt.shape[:2]
# comp_dict_topo_simple=get_comp_dict_topo_simple(width=width,height=height,level1=V_threshold,level2=0)
# x_values,y_values,pid_values=pdict.get_current_locations()
inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
img=inVc[...,0];dimgdt=dVcdt[...,0]

#initialize system
pdict=ParticlePBCDict(dict_tips=dict_topo, width=width, height=width)#, **kwargs)
t_prev=t;txt_prev=txt.copy()

t_death_lst.append(t_prev+dt)


#slow high http://localhost:8888/notebooks/dev%20measure%20bd%20events%20recursively.ipynb#precision method
# txt_prev,t_prev,min_sigma_max=find_stopping_point(dt, pdict, txt_prev, t_prev, save_every_n_frames=1)
#fast low precision method
txt_prev,t_prev,min_sigma_max=find_stopping_point(dt,pdict,txt_prev,t_prev,save_every_n_frames=100,V_threshold=V_threshold,pid_pair=pid_pair)

termination event found at time t=245.79999999997546, where dt=0.01 and L=200...


KeyError: 'lesser_arclen_lst'

In [ ]:
#TODO: init txt
txt_fn=f'{nb_dir}/Data/test_data/ic008.33_t_218.8.npz'
t=218.8
dt=0.01
txt=load_buffer(txt_fn)
inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
width,height=txt.shape[:2]
print(txt.shape)



txt_prev,t_prev,min_sigma_max=find_stopping_point(dt,pdict,txt_prev,t_prev,save_every_n_frames=100,V_threshold=V_threshold,pid_pair=pid_pair_lst[1])
t_death_lst.append(t_prev+dt)

In [ ]:
beep(6)

In [ ]:
#TODO: test that the tip order is deterministic
#TODO: enforce a deterministic tip order by sorting by x_values before returning tip locations 
# HINT: enforce (at the lowest level)


In [ ]:
#TODO: init txt
txt_fn=f'{nb_dir}/Data/test_data/ic008.33_t_218.8.npz'
t=218.8
dt=0.01
txt=load_buffer(txt_fn)
inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
width,height=txt.shape[:2]
print(txt.shape)

#TODO: recursively zoom into the first death up to atol

#TODO: repeat while there are still pid_pairs to zoom into
